In [1]:
import numpy as np
import pandas as pd
import pyomo.environ as pyo
from measure_optimize import MeasurementOptimizer, DataProcess 
import matplotlib.pyplot as plt

## Input Setting

### All measurements and index

- ads.gas_inlet.F (0)
- ads.gas_outlet.F (1)
- ads.gas_outlet.T (2)
- ads.gas_outlet.z (3)
- des.gas_inlet.F (4)
- des.gas_outlet.F (5)
- des.gas_outlet.T (6)
- des.gas_outlet.z (7)
- ads.T19 (8)
- ads.T23 (9)
- ads.T28 (10)
- ads.z19 (11)
- ads.z23 (12)
- ads.z28 (13)

One-time cost ones: 0,1,2, 4,5,6, 8,9,10

Time-cost ones: 3,7,11,12,13

In [2]:
all_names = ['Ads.gas_inlet.F', 'Ads.gas_outlet.F', 'Ads.gas_outlet.T', 
             'Ads.gas_outlet.z("CO2")', 'Des.gas_inlet.F', 'Des.gas_outlet.F', 
             'Des.gas_outlet.T', 'Des.gas_outlet.z("CO2")', 'Ads.T_g.Value(19,10)', 
             'Ads.T_g.Value(23,10)', 'Ads.T_g.Value(28,10)', 'Ads.z("CO2",19,10)', 
             'Ads.z("CO2",23,10)', 'Ads.z("CO2",28,10)']

In [3]:
Nt = 22
static_ind = [0,1,2,4,5,6,8,9,10]
dynamic_ind = [3,7,11,12,13]

In [4]:
dataObject = DataProcess()
dataObject.read_jaco('./RotaryBed/Q'+str(Nt)+'_scale.csv')
Q = dataObject.stack_Q(static_ind, dynamic_ind, Nt)


jacobian shape: (308, 5)


In [5]:
static_name = [all_names[i] for i in static_ind]
dynamic_name = [all_names[i] for i in dynamic_ind]
static_idx_upd = [0,1,2,3,4,5,6,7,8]
dynamic_idx_upd = [9,10,11,12,13]

num_param = 5

measure_names = static_name + dynamic_name
print(measure_names)

['Ads.gas_inlet.F', 'Ads.gas_outlet.F', 'Ads.gas_outlet.T', 'Des.gas_inlet.F', 'Des.gas_outlet.F', 'Des.gas_outlet.T', 'Ads.T_g.Value(19,10)', 'Ads.T_g.Value(23,10)', 'Ads.T_g.Value(28,10)', 'Ads.gas_outlet.z("CO2")', 'Des.gas_outlet.z("CO2")', 'Ads.z("CO2",19,10)', 'Ads.z("CO2",23,10)', 'Ads.z("CO2",28,10)']


In [6]:
calculator = MeasurementOptimizer(Q, static_idx_upd, dynamic_idx_upd, 
                                  num_param, measure_names=measure_names, verbose=True)

fim_expect = calculator.fim_computation()

print(np.shape(calculator.fim_collection))

(14161, 5, 5)


In [7]:
#print(calculator.dynamic_to_flatten[1])

### Set up grey-box 

In [8]:
from greybox_generalize import LogDetModel

In [9]:
def build_model_external(m):
    ex_model = LogDetModel(num_para=5)
    m.egb = ExternalGreyBoxBlock()
    m.egb.set_external_model(ex_model)

### Reform the inputs

In [10]:
num_static = len(static_ind)
num_dynamic  = len(dynamic_ind)
num_total = num_static + num_dynamic*Nt

In [11]:
cost = [1000, #ads.gas_inlet.F (0)
        1000, #ads.gas_outlet.F (1)
         500, #ads.gas_outlet.T (2)
        1000, #des.gas_inlet.F (4)
        1000, #des.gas_outlet.F (5)
         500, #des.gas_outlet.T (6)
         1000, #ads.T19 (8)
         1000, #ads.T23 (9)
         1000 #ads.T28 (10)
       ]

for i in range(Nt*num_dynamic):
    cost.append(100)

install_cost_dynamic = [100, 100, 500, 500, 500]

In [12]:
init_y = np.random.rand(num_total-num_static)

update_y = [1]*num_static 

for i in init_y:
    if i > 0.5: 
        update_y.append(1)
    else:
        update_y.append(0)

init_cov_y = [[0]*num_total for i in range(num_total)]

for j in range(num_total):
    for k in range(num_total):
        if j==k:
            init_cov_y[j][k] = update_y[k]
            
        else:
            init_cov_y[j][k] = min(update_y[k], update_y[j])

fim_prior = np.identity((5))
print(fim_prior)

[[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]


In [13]:
num_fixed_opt = 9

num_dynamic_opt = 5

mip_option = False
objective = "A"
sparse_opt = True
fix_opt = False

each_manual_num = 5
total_manual_num = 20
budget_opt = 10000  

discretize_time = 10
num_dynamic_time = np.linspace(2,220,Nt)
    
#print(num_dynamic_time)

dynamic_time_dict = {}
for i, tim in enumerate(num_dynamic_time):
    dynamic_time_dict[i] = tim 
    
#print(dynamic_time_dict)

In [14]:
mod = calculator.continuous_optimization(cost, mixed_integer=mip_option, 
                  obj=objective, fix=fix_opt, sparse=sparse_opt, 
                    num_dynamic_t_name = num_dynamic_time, 
                    discretize_time = discretize_time, 
                    manual_number = total_manual_num, budget=budget_opt,
                    dynamic_install_cost = install_cost_dynamic, 
                    each_manual_number = each_manual_num,  initial_fim = fim_prior)

mod = calculator.solve(mod, mip_option=mip_option, objective = objective)

Ipopt 3.13.2: linear_solver=ma57


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        compu

In iteration 72, 1 Slack too small, adjusting variable bound
  73 -3.2681963e+04 2.18e-11 7.61e+00  -9.0 1.32e-04    -  1.00e+00 5.03e-01f  1
  74 -3.2681963e+04 2.18e-11 1.41e+00  -9.0 6.89e-05    -  1.00e+00 8.21e-01h  1
  75 -3.2681963e+04 1.46e-11 1.29e-11  -9.0 9.34e-06    -  1.00e+00 1.00e+00f  1

Number of Iterations....: 75

                                   (scaled)                 (unscaled)
Objective...............:  -3.2681962604851760e+04   -3.2681962604851760e+04
Dual infeasibility......:   1.2926580812699928e-11    1.2926580812699928e-11
Constraint violation....:   9.0949470177292824e-13    1.4551915228366852e-11
Complementarity.........:   2.4548993514921702e-09    2.4548993514921702e-09
Overall NLP error.......:   2.4548993514921702e-09    2.4548993514921702e-09


Number of objective function evaluations             = 76
Number of objective gradient evaluations             = 76
Number of equality constraint evaluations            = 76
Number of inequality constraint e

In [15]:
fim_result = np.zeros((5,5))
for i in range(5):
    for j in range(i,5):
        fim_result[i,j] = fim_result[j,i] = pyo.value(mod.TotalFIM[i,j])
        
print(fim_result)  
print('trace:', np.trace(fim_result))
print('det:', np.linalg.det(fim_result))
print(np.linalg.eigvals(fim_result))

print("Pyomo OF:", pyo.value(mod.Obj))
print("Log_det:", np.log(np.linalg.det(fim_result)))

[[ 5.41525774e+03  1.02442612e+00 -4.15244981e+03  1.08772536e+04
  -3.50853226e+02]
 [ 1.02442612e+00  9.75318387e-01 -1.36974325e-01  9.74933747e+00
  -1.58012639e+00]
 [-4.15244981e+03 -1.36974325e-01  3.29923027e+03 -8.51763311e+03
   1.80468545e+02]
 [ 1.08772536e+04  9.74933747e+00 -8.51763311e+03  2.38294651e+04
  -5.83373454e+02]
 [-3.50853226e+02 -1.58012639e+00  1.80468545e+02 -5.83373454e+02
   1.37034206e+02]]
trace: 32681.96260485176
det: 44535061831.72334
[3.20159706e+04 4.77958777e+02 1.68042121e+02 1.90835917e+01
 9.07541881e-01]
Pyomo OF: 32681.96260485176
Log_det: 24.51954262221422


In [16]:
ans_y, sol_y = calculator.extract_solutions(mod)

Ads.gas_inlet.F :  1.0
Ads.gas_outlet.F :  0.45
Ads.gas_outlet.T :  0.45
Des.gas_inlet.F :  0.45
Des.gas_outlet.F :  0.45
Des.gas_outlet.T :  0.45
Ads.T_g.Value(19,10) :  1.0
Ads.T_g.Value(23,10) :  0.45
Ads.T_g.Value(28,10) :  1.0
Ads.gas_outlet.z("CO2") (time points [min]):
157.71428571428572
178.47619047619048
188.85714285714286
199.23809523809524
220.0
Des.gas_outlet.z("CO2") (time points [min]):
53.904761904761905
74.66666666666667
85.04761904761905
95.42857142857143
105.80952380952381
Ads.z("CO2",19,10) (time points [min]):
12.380952380952381
33.142857142857146
53.904761904761905
74.66666666666667
157.71428571428572
Ads.z("CO2",23,10) (time points [min]):
95.42857142857143
136.95238095238096
220.0
Ads.z("CO2",28,10) (time points [min]):
53.904761904761905
199.23809523809524


In [18]:
# verify cost 
real_cost = 0 
for i in range(num_total):
    real_cost += ans_y[i,i]*cost[i]
    
dynamic_inst = [0]*num_dynamic
for j in range(num_dynamic):
    for t in range(Nt):
        if sol_y[j,t]>0.99:
            dynamic_inst[j] = 1 
            
real_cost += sum(dynamic_inst[i]*install_cost_dynamic[i] for i in range(num_dynamic))
print(dynamic_inst)
print(real_cost)

[1, 1, 1, 1, 1]
8950.0
